In [ ]:
pip install azure-identity

In [ ]:
pip install openai

This notebooks configures Azure OpenAI Python SDK to use Service Principal in Azure Active Directory and uses Token authentication rather than using Open AI access key. 

This notebook can be run on Databricks with secret scope configured. The Service Principal ID(Client Id), Client Secret and Azure Active Directory Tenant Id can be created in databricks backed vault and retrieved at runtime and stored in environment variables. Azure OpenAI python SDK automatically detects the environment variables and acquires the AAD token. 

The Service Principal needs to be granted "Cognitive Services User" role in Azure portal to the Azure Resource Group which contains the Azure OpenAI Resource or use the below Azure CLI commands.

    openAIRg="<OpenAI Resource Group Name>
    spId="<Service Principal GUID>"
    export resourceId=$(az group show -g $openAIRg | jq -r .id)
    az role assignment create --role "Cognitive Services User" --assignee $spId --scope $resourceId

In [ ]:

import os

# Specify the name of the secret scope and key vault secret name
scope_name = "akv"

# Get the secret value
OPENAI_APP_ID = dbutils.secrets.get(scope=scope_name, key="openai-app-id") # replace key name with actual key name from key vault
OPENAI_APP_SECRET = dbutils.secrets.get(scope=scope_name, key="openai-app-secret") # replace key name with actual key name from key vault
TENANT_ID = dbutils.secrets.get(scope=scope_name, key="openai-tenantid") # replace key name with actual key name from key vault

# Use the secret value in your code
print(OPENAI_APP_ID)
print(OPENAI_APP_SECRET)
print(TENANT_ID)

os.environ["AZURE_TENANT_ID"] = TENANT_ID
os.environ["AZURE_CLIENT_ID"] = OPENAI_APP_ID
os.environ["AZURE_CLIENT_SECRET"] = OPENAI_APP_SECRET


Option 1: Use DeviceCodeCredential to login interactively using user token

In [ ]:
from azure.identity import DeviceCodeCredential
import os

import openai
device_credential = DeviceCodeCredential(tenant_id="0fbe7234-45ea-498b-b7e4-1a8b2d3be4d9")
token = device_credential.get_token("https://cognitiveservices.azure.com/.default")

openai.api_type = 'azure_ad'
openai.api_key = token.token

Option 2: Use DefaultAzureCredential to acquire token using AAD Service Principal

In [ ]:
from azure.identity import DefaultAzureCredential
import openai

# Request credential
default_credential = DefaultAzureCredential()
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# Setup parameters
openai.api_type = "azure_ad"
openai.api_key = token.token
openai.api_base = "https://anildwaopenai2.openai.azure.com/"
openai.api_version = "2023-03-15-preview"

In [ ]:
completion = openai.ChatCompletion.create(deployment_id="chatgpt", messages=[{"role": "user", "content": "Hello world!"}])
print(completion.choices[0].message.content)